Upgraded version with multiple tools

In [1]:
from langchain_cohere import ChatCohere
from dotenv import load_dotenv
from langchain_community.tools import DuckDuckGoSearchRun
from langchain_classic.agents import create_react_agent, AgentExecutor
from langchain_classic import hub
from langchain_core.tools import tool
import requests

In [2]:
# Load environment variables
load_dotenv()

# Initialize LLM
llm = ChatCohere(model="command-r-plus-08-2024")

In [3]:
# Initialize search tool
search_tool = DuckDuckGoSearchRun()

In [4]:
@tool
def get_weather_data(city:str)->str:
    """Fetches the current weather data for a given city.
    
    Args:
        city: Name of the city (e.g., 'Bhopal', 'Delhi', 'Mumbai',"Nepal")
    
    Returns:
        str: Weather information including temperature, description, and conditions
    """
    url = f"https://api.weatherstack.com/current?access_key=dd911bf4b170b7e2cb05d2f157089251&query={city}"
    response=requests.get(url=url)
    return response.json()


In [5]:
get_weather_data.invoke("Bhubaneswar")

{'request': {'type': 'City',
  'query': 'Bhubaneswar, India',
  'language': 'en',
  'unit': 'm'},
 'location': {'name': 'Bhubaneswar',
  'country': 'India',
  'region': 'Orissa',
  'lat': '20.233',
  'lon': '85.833',
  'timezone_id': 'Asia/Kolkata',
  'localtime': '2025-11-27 12:09',
  'localtime_epoch': 1764245340,
  'utc_offset': '5.50'},
 'current': {'observation_time': '06:39 AM',
  'temperature': 25,
  'weather_code': 143,
  'weather_icons': ['https://cdn.worldweatheronline.com/images/wsymbols01_png_64/wsymbol_0006_mist.png'],
  'weather_descriptions': ['Haze'],
  'astro': {'sunrise': '06:03 AM',
   'sunset': '05:05 PM',
   'moonrise': '11:29 AM',
   'moonset': '11:05 PM',
   'moon_phase': 'Waxing Crescent',
   'moon_illumination': 37},
  'air_quality': {'co': '701.85',
   'no2': '14.55',
   'o3': '59',
   'so2': '32.35',
   'pm2_5': '68.35',
   'pm10': '70.25',
   'us-epa-index': '4',
   'gb-defra-index': '4'},
  'wind_speed': 13,
  'wind_degree': 40,
  'wind_dir': 'NE',
  'press

In [6]:
# Pull the React prompt from LangChain Hub
prompt = hub.pull("hwchase17/react")

In [7]:
# Create agent
agent = create_react_agent(
    llm=llm,
    tools=[search_tool,get_weather_data],
    prompt=prompt
)

In [8]:
# Create agent executor
agent_executor = AgentExecutor(
    agent=agent,
    tools=[search_tool,get_weather_data],
    verbose=True,
    handle_parsing_errors=True
)

In [9]:
# Invoke agent
response = agent_executor.invoke({"input": "Find the capital of Madhya Pradesh,then find its current weather condition"})

# Print response
print(response)



> Entering new AgentExecutor chain...
Thought: First, I need to find the capital of Madhya Pradesh. Once I have that information, I can then search for the current weather conditions in that city.
Action: duckduckgo_search
Action Input: capital of Madhya PradeshOIndore (/ ɪnˈdɔːr / ⓘ; ISO: Indaura, Hindi: [ɪn̪d̪ɔːr]) is the largest and most populous city in the Indian state of Madhya Pradesh. The commercial capital of the state, it has been declared as the cleanest city in India 8 years in a row. [15] It is also considered the largest education hub in central India and houses campuses of both the Indian Institute of Technology and the ... Bombay State was enlarged by the addition of Saurashtra State and Kutch State, the Marathi-speaking districts of Nagpur division of Madhya Pradesh and the Marathwada region of Hyderabad State. Its capital is Bhopal, and Madhya Pradesh is the second-largest Indian state by area and the fifth-largest state by population with over 72 million residents.

Most Basic way to create agent

In [1]:
from langchain_cohere import ChatCohere
from dotenv import load_dotenv
from langchain_community.tools import DuckDuckGoSearchRun
from langchain.agents import create_agent

# Load environment variables
load_dotenv()

# Initialize LLM
llm = ChatCohere(model="command-r-plus-08-2024")

# Initialize search tool
search_tool = DuckDuckGoSearchRun()

# Create agent (no agent_type needed)
agent_executor = create_agent(
    model=llm,
    tools=[search_tool]
)

# Invoke agent
response = agent_executor.invoke({"messages": [("user", "Winner of last IPL match")]})

# Print response
print(response)

# Print just the final answer
print("\n=== Final Answer ===")
print(response['messages'][-1].content)

{'messages': [HumanMessage(content='Winner of last IPL match', additional_kwargs={}, response_metadata={}, id='6c312567-e8dd-4b0f-b166-8bb4f19a4411'), AIMessage(content="I will search for 'winner of last IPL match'.", additional_kwargs={'id': '41bb83c9-8bcb-4eac-a034-22c444745862', 'finish_reason': 'TOOL_CALL', 'tool_plan': "I will search for 'winner of last IPL match'.", 'tool_calls': [{'id': 'duckduckgo_search_xedc4xg1hs61', 'type': 'function', 'function': {'name': 'duckduckgo_search', 'arguments': '{"query":"winner of last IPL match"}'}}], 'token_count': {'input_tokens': 920.0, 'output_tokens': 59.0}}, response_metadata={'id': '41bb83c9-8bcb-4eac-a034-22c444745862', 'finish_reason': 'TOOL_CALL', 'tool_plan': "I will search for 'winner of last IPL match'.", 'tool_calls': [{'id': 'duckduckgo_search_xedc4xg1hs61', 'type': 'function', 'function': {'name': 'duckduckgo_search', 'arguments': '{"query":"winner of last IPL match"}'}}], 'token_count': {'input_tokens': 920.0, 'output_tokens': 